<a href="https://colab.research.google.com/github/naveedkhalid091/Learn_Agentic_AI/blob/main/step02_generative_ai_for_beginners/02(b)_RAG_implementation_with_PineconeDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## RAG Implementation with PineConeDB:

**Step#1:** Install the necessary environment as follows:  

In [6]:
!pip install -qU langchain-pinecone langchain-google-genai

**Step#2:** In this step we need a to initialize the Pinecone Database.

So go to the Pinecone Database, create your account and Generate API key.    



In [7]:
from pinecone import Pinecone, ServerlessSpec
from google.colab import userdata
pinecone_api_key = userdata.get('PINECONE_API_KEY')

pc = Pinecone(api_key=pinecone_api_key)

Step#3 - Create Indexing in the Pinecone Database:

The below code can also be taken from the [PineCone Vector Database](https://www.pinecone.io/). First `Sign-in` and then go to the `Database -> Indexes` section.

You can also see the below code in the [PineCone Documentation](https://python.langchain.com/docs/integrations/vectorstores/pinecone/) by scrolling down to the Initialization Section.

In [8]:
import time

index_name = "online-rag-project"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

pc.create_index(
        name=index_name,
        dimension=768, # Replace with your model dimensions
        metric="cosine", # Replace with your model metric
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)


PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': 'd5fab04aeb46b0098cfae5473b013cdd', 'Date': 'Sat, 04 Jan 2025 13:00:05 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [16]:
index = pc.Index(index_name)

Now our Pinecone index is sucessfully setup, You can also varify this created Index (`online-rag-project`) into the PineCone Database and navigate to `Database -> Indexes`. After creating index, now all data will be stored in this `Index`.  


Now, we need to select the embedding model as we had studied that the data can only be saved in vector form so we need to select the embedding model (which will convert our data into vectors/numbers).

We will select the google embeddings model in our workings:

You can find the Google embedding model from the below mentioned documentation.

- [Google Embedding Documentation](https://python.langchain.com/docs/integrations/text_embedding/google_generative_ai/).


This these documentations It is clearly mentioned that you can Connect to Google's generative AI embeddings service using the `GoogleGenerativeAIEmbeddings` class, found in the `langchain-google-genai package` which we have installed at step # 1.

But keep in your mind that You also need to get access of Gemini model for accessing the embedding model.

So first access the Gemini model and then access the embeddings model as below:






In [9]:
import os

from google.colab import userdata
GOOGLE_API_KEY='GOOGLE_API_KEY'
os.environ[GOOGLE_API_KEY]=userdata.get('GOOGLE_API_KEY')



Now embed the model as below:  



**Note:** This below code is taken from [Google Embedding Documentation](https://python.langchain.com/docs/integrations/text_embedding/google_generative_ai/), from the `usage` section.

In [10]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector = embeddings.embed_query("hello, world!")
print(vector[:5])

[0.05168594419956207, -0.030764883384108543, -0.03062233328819275, -0.02802734263241291, 0.01813093200325966]


The next step is to load the documents into the vector database and convert them into embeddings. LangChain simplifies this process with its vector store, which provides a unified interface for various vector databases.

This flexibility allows developers to switch between databases with ease by specifying the database name, configuring it, and making minor adjustments to the code.

First Import the vector store as:

In [17]:
from langchain_pinecone import PineconeVectorStore

vector_store= PineconeVectorStore(index=index, embedding=embeddings)

Now firstly we will save the data and secondly we will try to retrieve it.